In [ ]:
# Import necessary libraries
import  rdflib
from    rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS, OWL, Literal, URIRef
from    rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO, RDFS
import pyvis
from pyvis.network import Network
from IPython.display import IFrame

# 1. Load the Turtle file into an RDF graph.
# Replace 'your_file.ttl' with the path to your Turtle file.
g = Graph()

rdf     = rdflib.Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs    = rdflib.Namespace("http://www.w3.org/2000/01/rdf-schema#")
gr      = rdflib.Namespace("http://purl.org/goodrelations/v1#")
owl     = rdflib.Namespace("http://www.w3.org/2002/07/owl#")

dbo     = rdflib.Namespace("http://dbpedia.org/ontology/")
dbr     = rdflib.Namespace("http://dbpedia.org/resource/")

g.bind("schema", SDO)
g.bind("dbo",dbo)
g.bind("dbr",dbr)
g.bind("rdf", rdf)
g.bind("rdfs", rdfs)
g.bind("gr", gr)
g.bind("owl", owl)
g.bind("xsd", XSD)


SDO = Namespace("https://schema.org/")
g.bind("schema", SDO)

g.parse("final.ttl", format="turtle")
print("Turtle file loaded. The graph has {} triples.".format(len(g)))

In [ ]:
#4.1
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?name ?inName
WHERE {
    ?recipe a schema:Recipe ;
      schema:hasPart ?ingredient ;
      schema:name ?name .
    ?ingredient schema:name ?inName .
    FILTER (str(?inName) = "mango")
}
"""
qResult = g.query(query)

for row in qResult:
    print(f" {row.name} {row.inName}")

In [ ]:
#4.2
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?name
WHERE {
  ?recipe a schema:Recipe ;
      schema:name ?name;
      schema:keywords "Healty";
      schema:recipeCategory "Pie" ;
      schema:cookTime ?cooktime ;
  FILTER( ?cooktime < (2 * 3600) )
}
"""
qResult = g.query(query)

for row in qResult:
    print(f"  {row.name} ")

In [ ]:
#4.3
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?resturant ?name
WHERE {
  ?resturant a schema:Restaurant ;
      schema:name ?name;
      schema:City "New Delhi";
      schema:servesCuisine "Chinese";
      schema:DeliveryMethod "1" .
}
"""
qResult = g.query(query)

for row in qResult:
    print(f" {row.resturant} {row.name} ")

In [ ]:
#4.4
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT (AVG(?price) AS ?avgPrice)
WHERE {
  ?resturant a schema:Restaurant ;
      schema:name ?name;
      schema:City "Davenport";
      schema:servesCuisine "Asian"; 
      foodnutrition:costForTwo ?price .
      BIND( (xsd:double(?price)) AS ?price)
}
"""
#find a list of all asian cusines
qResult = g.query(query)

for row in qResult:
    print(f" {row.avgPrice} ")

In [ ]:
#4.5
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?name ?img 
WHERE {
  ?recipe a schema:Recipe ;
        schema:name ?name;
        schema:image ?img ;
        schema:keywords "Easy";
        schema:recipeCategory "Dessert" ;
        schema:datePublished ?date .
    ?nutrition a schema:NutritionInformation;
        schema:name ?name;
        schema:calories ?cal .
    FILTER (?date >= "1990-01-01"^^xsd:date)
    FILTER (str(?img) != "character(0)") .
} 
ORDER BY (?cal)
Limit 5
"""
qResult = g.query(query)

for row in qResult:
    print(f" {row.name} {row.img}")

In [ ]:
#4.6
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT Distinct ?name ?prepTime ?sugar
WHERE {
  ?recipe a schema:Recipe ;
      schema:name ?name ;
      schema:prepTime ?prepTime;
      schema:recipeCategory "Beverages" .
  ?nutrition a schema:NutritionInformation;
        schema:name ?name;
        schema:sugarContent ?sugar 
}
LIMIT 10
"""
qResult = g.query(query)

for row in qResult:
    print(f"  {row.name} {row.prepTime} {row.sugar}")

In [ ]:
#4.7
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?name ?resturantName
WHERE {
  ?recipe a schema:Recipe ;
      schema:name ?name ;
      schema:servesCuisine ?cuisine.
  ?nutrition a schema:NutritionInformation;
      schema:name ?name;
      schema:proteinContent ?protein.
  ?resturant a schema:Restaurant;
    schema:name ?resturantName ;
    schema:servesCuisine ?ResCuisine;
    schema:Country "UK" .
  filter contains(?cuisine, ?ResCuisine)
}
ORDER BY (?protein)
LIMIT 10
"""
qResult = g.query(query)

for row in qResult:
    print(f"  {row.name} {row.resturantName}")

In [ ]:
#4.8
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>


SELECT  ?name ?resturantName ?aggRating
WHERE {
    ?recipe a schema:Recipe ;
        schema:name ?name ;
        schema:servesCuisine ?cuisine.
    ?nutrition a schema:NutritionInformation;
        schema:name ?name;
        schema:proteinContent ?protein;
        schema:fiberContent ?fiber;
        schema:sugarContent ?sugar.
    ?resturant a schema:Restaurant;
      schema:name ?resturantName ;
      schema:servesCuisine ?ResCuisine;
      schema:aggregateRating ?aggRating.
    BIND( (xsd:double(?protein)) AS ?protein)
    BIND( (xsd:double(?fiber)) AS ?fiber)
    BIND( (xsd:double(?sugar)) AS ?sugar)
    filter contains(?cuisine, ?ResCuisine)

    }
    ORDER BY (?protein + (1.5 * ?fiber) - (2 * ?sugar))
    LIMIT 5
"""
qResult = g.query(query)

for row in qResult:
    print(f"  {row.name} {row.resturantName} {row.aggRating}")